In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
from dataset import *
from model import *
import os
import SimpleITK as sitk
import math
%matplotlib widget

In [2]:
mode='gpu'

In [3]:
if mode=='gpu':
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # after switch device, you need restart the kernel
#     torch.cuda.set_device(1)
    torch.set_default_tensor_type('torch.cuda.DoubleTensor')
else:
    device = torch.device('cpu')
    torch.set_default_dtype(torch.float64)

## Testing
### initialization

In [4]:
epoch = 262
output_dir = 'Models/UnetLarge'
checkpoint = torch.load(f'{output_dir}/epoch_{epoch}_checkpoint.pth.tar')
model = UNet256()

model.load_state_dict(checkpoint['model_state_dict'])
net = torch.nn.DataParallel(model, device_ids=[0, 1])

# params 7511338, # conv layers 46


In [6]:
case_info = []
root_dir = '/home/sci/hdai/Projects/Dataset/LymphNodes'
patch_size = 128
field_list = ['Series UID', 'Collection', '3rd Party Analysis', 
                      'Data Description URI', 'Subject ID', 'Study UID', 
                      'Study Description', 'Study Date', 'Series Description', 
                      'Manufacturer', 'Modality', 'SOP Class Name', 
                      'SOP Class UID', 'Number of Images', 'File Size', 
                      'File Location', 'Download Timestamp']
with open(f'{root_dir}/metadata.csv', mode='r') as infile:
    reader = csv.reader(infile)
    for row in reader:
        case_info.append({field_list[i]:row[i] for i in range(len(row))})
case_info.pop(0)
        
for case in tqdm(case_info):         
#         construct 3d CT from dicom folder
        # '/CT Lymph Nodes/ABD_LYMPH_003/09-14-2014-ABDLYMPH003-abdominallymphnodes-39052/abdominallymphnodes-65663'
    relative_ct_folder_path = case['File Location'][1:].replace('\\','/')
        # '/home/sci/hdai/Projects/LymphNodes/CT Lymph Nodes/ABD_LYMPH_003/09-14-2014-ABDLYMPH003-abdominallymphnodes-39052/abdominallymphnodes-65663'
    ct_folder_path = f'{root_dir}{relative_ct_folder_path}'
    slice_name_list = [f for f in os.listdir(ct_folder_path)]
    slice_name_list.sort()
    slice_list = []
    for slice_name in slice_name_list:
        ds = pd.dcmread(f'{ct_folder_path}/{slice_name}')
        slice_list.append(torch.from_numpy(ds.pixel_array.transpose()))
    img = torch.stack(slice_list,-1).to(device)
        
#     centroid_x = 256#int(torch.mean(idx_x.float()))
#     centroid_y = 256#int(torch.mean(idx_y.float()))
#     centroid_z = 300#int(torch.mean(idx_z.float()))
# #         load 3d mask
#     half_patch_size = int(patch_size/2)
#     img = img[centroid_x-half_patch_size:centroid_x+half_patch_size, centroid_y-half_patch_size:centroid_y+half_patch_size, centroid_z-half_patch_size:centroid_z+half_patch_size]
#     mask_pred = model(img.unsqueeze(0).unsqueeze(0))
    case_name = case['File Location'][17:30].replace('\\','/')
    
    segment_depth = 128
    segment_num = math.ceil(img.shape[2]/segment_depth)
    mask_pred_segment_list = []
    for i in range(segment_num):
        begin = i*segment_depth
        end = min(i*segment_depth+segment_depth,img.shape[2])
        mask_pred_segment = model(img[192:320,192:320,begin:end].unsqueeze(0).unsqueeze(0))
        mask_pred_segment_list.append(mask_pred_segment)
        
    mask_pred = torch.stack(mask_pred_segment_list,-1)
#     print(case_name)
    mask_path = f'{output_dir}/PredResult/{case_name}_pred_mask.nii.gz'
    nib.save(nib.Nifti1Image(mask_pred[0,0].cpu().detach().numpy(), None), mask_path)

  0%|          | 0/174 [00:02<?, ?it/s]


RuntimeError: Given input size: (128x8x8x1). Calculated output size: (128x4x4x0). Output size is too small

In [ ]:
    relative_ct_folder_path = case['File Location'][1:].replace('\\','/')
        # '/home/sci/hdai/Projects/LymphNodes/CT Lymph Nodes/ABD_LYMPH_003/09-14-2014-ABDLYMPH003-abdominallymphnodes-39052/abdominallymphnodes-65663'
    ct_folder_path = f'{root_dir}{relative_ct_folder_path}'
    slice_name_list = [f for f in os.listdir(ct_folder_path)]
    slice_name_list.sort()
    slice_list = []
    for slice_name in slice_name_list:
        ds = pd.dcmread(f'{ct_folder_path}/{slice_name}')
        slice_list.append(torch.from_numpy(ds.pixel_array.transpose()))
    img = torch.stack(slice_list,-1).to(device)

In [ ]:
img.shape

In [ ]:
mask_pred_segment.shape

In [7]:
begin,end

(640, 661)